In [1]:
#!pip install opencv-contrib-python
!pip install --upgrade google-cloud-vision

In [2]:
import matplotlib.pyplot as plt

In [3]:
import os
import io
import numpy as np
import pandas as pd
import platform
from PIL import ImageFont, ImageDraw, Image
import cv2
from cv2 import imshow

In [5]:
import io
import os

# Imports the Google Cloud client library
from google.cloud import vision

# Instantiates a client
#client = vision.ImageAnnotatorClient()

In [6]:
from pytesseract import Output
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score

In [7]:
import pandas as pd
import numpy as np
import re
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pytesseract

In [8]:
def imshow(title='image', img=None, figsize=(8,5)):
    plt.figure(figsize=figsize)
    
    if type(img) == list:
        if type(title) == list:
            titles=title
        else:
            titles=[]
            
            for i in range(len(img)):
                titles.append(title)
                
        for i in range(len(img)):
            if len(img[i].shape)<=2:
                rgbImg=cv2.cvtColor(img[i], cv2.COLOR_GRAY2RGB)
            else:
                rgbImg=cv2.cvtColor(img[i], cv2.COLOR_BGR2RGB)
                
            plt.subplot(1, len(img), i+1), plt.imshow(rgbImg)
            plt.title(titles[i])
            plt.xticks([]), plt.yticks([])
            
        plt.show()
    else:
        if len(img.shape) <3:
            rgbImg=cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        else:
            rgbImg=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        plt.imshow(rgbImg)
        plt.title(title)
        plt.xticks([]), plt.yticks([])
        plt.show()

- 한글을 출력하는 경우 한글이 깨지는 문제를 해결하기 위한

In [9]:
def putText(image, text, x, y, color=(0, 255, 0), font_size=22):
    if type(image) == np.ndarray:
        color_coverted = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(color_coverted)
 
    if platform.system() == 'Darwin':
        font = 'AppleGothic.ttf'
    elif platform.system() == 'Windows':
        font = 'malgun.ttf'
        
    image_font = ImageFont.truetype(font, font_size)
    font = ImageFont.load_default()
    draw = ImageDraw.Draw(image)
 
    draw.text((x, y), text, font=image_font, fill=color)
    
    numpy_image = np.array(image)
    opencv_image = cv2.cvtColor(numpy_image, cv2.COLOR_RGB2BGR)
 
    return opencv_image

In [10]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'ocrproject-363109-b1fa9be9343f.json'
 
client_options = {'api_endpoint': 'eu-vision.googleapis.com'}
client = vision.ImageAnnotatorClient(client_options=client_options)

- 데이터불러오기

In [11]:
train_df=pd.read_csv('train.csv', encoding='utf-8')
test_df=pd.read_csv('test.csv', encoding='utf-8')
train_df

,img_path,text
0,./train/train_00001.png,골목미용실
1,./train/train_00002.png,한성부동산
2,./train/train_00003.png,홍라운지
3,./train/train_00004.png,모단걸응접실
4,./train/train_00005.png,씨앗양식
...,...,...
12154,./train/train_12161.png,넬슨
12155,./train/train_12162.png,앤드루 램버트 지음
12156,./train/train_12163.png,박아람 옮김
12157,./train/train_12164.png,판결의


In [ ]:
#Load Image(byte array로 변환하여 로드)
result=[]
#result.append(text)

for path in test_df['img_path'][:2]:
    #path = train_df['img_path'].values[1]

#path = train_df['img_path'].values[1]
    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    
#API 호출
    image = vision.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations   


#결과 확인
    img = cv2.imread(path)
    roi_img = img.copy()
    
    for text in texts:
        print('\n"{}"'.format(text.description))
 
        vertices = (['({},{})'.format(vertex.x, vertex.y)
                for vertex in text.bounding_poly.vertices])
    
        ocr_text = text.description
        result.append(ocr_text)
        #x1 = text.bounding_poly.vertices[0].x
        #y1 = text.bounding_poly.vertices[0].y
        #x2 = text.bounding_poly.vertices[1].x
        #y2 = text.bounding_poly.vertices[2].y
    
        #cv2.rectangle(roi_img, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)
        #roi_img = putText(roi_img, ocr_text, x1, y1 - 30, font_size=30)
 
    #if response.error.message:
        #raise Exception(
            #'{}\nFor more info on error messages, check: '
            #'https://cloud.google.com/apis/design/errors'.format(
                #response.error.message))
    
#imshow(["Original", "ROI"], [img, roi_img], figsize=(16, 10))
#imshow("Original", img,  figsize=(16, 10))
#imshow("ROI", roi_img, figsize=(16, 10))

In [ ]:
#Load Image(byte array로 변환하여 로드)
result=[]
#result.append(text)

for path in test_df['img_path']:

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations   
    #print(texts[0].description)
    try:
        text=texts[0].description
    except:
        text = ''
   
    text = text.replace(' ', '')
    text = text.replace('\n', '')
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', text)
    text = text.strip()
    print(test_df.index, text)
    result.append(text)

RangeIndex(start=0, stop=3669, step=1) 起
RangeIndex(start=0, stop=3669, step=1) 这
RangeIndex(start=0, stop=3669, step=1) 2
RangeIndex(start=0, stop=3669, step=1) 풍경
RangeIndex(start=0, stop=3669, step=1) 덕천중학교
RangeIndex(start=0, stop=3669, step=1) 로뎀노인요양원
RangeIndex(start=0, stop=3669, step=1) 한우리건축사사무소
RangeIndex(start=0, stop=3669, step=1) 배꼽시계
RangeIndex(start=0, stop=3669, step=1) 의원
RangeIndex(start=0, stop=3669, step=1) 소주
RangeIndex(start=0, stop=3669, step=1) 신선육국내산
RangeIndex(start=0, stop=3669, step=1) 피엘라벤진주첩
RangeIndex(start=0, stop=3669, step=1) 수족관
RangeIndex(start=0, stop=3669, step=1) 네
RangeIndex(start=0, stop=3669, step=1) 
RangeIndex(start=0, stop=3669, step=1) 성원유리가공사
RangeIndex(start=0, stop=3669, step=1) 다정
RangeIndex(start=0, stop=3669, step=1) 미용실
RangeIndex(start=0, stop=3669, step=1) 왕배화로
RangeIndex(start=0, stop=3669, step=1) 시카고뽑
RangeIndex(start=0, stop=3669, step=1) ILE강남장연관
RangeIndex(start=0, stop=3669, step=1) 제민내
RangeIndex(start=0, stop=3669, step=1)

RangeIndex(start=0, stop=3669, step=1) 김선영화실
RangeIndex(start=0, stop=3669, step=1) 태성
RangeIndex(start=0, stop=3669, step=1) 세종타운
RangeIndex(start=0, stop=3669, step=1) 세종씨엠에스
RangeIndex(start=0, stop=3669, step=1) 해드림하우스
RangeIndex(start=0, stop=3669, step=1) 다나
RangeIndex(start=0, stop=3669, step=1) 전민마트
RangeIndex(start=0, stop=3669, step=1) 바0O
RangeIndex(start=0, stop=3669, step=1) 크로스빌전문점
RangeIndex(start=0, stop=3669, step=1) 수민이
RangeIndex(start=0, stop=3669, step=1) 나
RangeIndex(start=0, stop=3669, step=1) EITHL
RangeIndex(start=0, stop=3669, step=1) 노래실
RangeIndex(start=0, stop=3669, step=1) 원컴퓨터
RangeIndex(start=0, stop=3669, step=1) 모바일
RangeIndex(start=0, stop=3669, step=1) 유성닭발
RangeIndex(start=0, stop=3669, step=1) 백그릇
RangeIndex(start=0, stop=3669, step=1) HILIGIO버닝헤어
RangeIndex(start=0, stop=3669, step=1) 귀족
RangeIndex(start=0, stop=3669, step=1) 보리밥
RangeIndex(start=0, stop=3669, step=1) 南立名便
RangeIndex(start=0, stop=3669, step=1) 탱고
RangeIndex(start=0, stop=3669, st

RangeIndex(start=0, stop=3669, step=1) 이층
RangeIndex(start=0, stop=3669, step=1) 그리고
RangeIndex(start=0, stop=3669, step=1) 맥주
RangeIndex(start=0, stop=3669, step=1) 초밥
RangeIndex(start=0, stop=3669, step=1) 
RangeIndex(start=0, stop=3669, step=1) 방
RangeIndex(start=0, stop=3669, step=1) 760
RangeIndex(start=0, stop=3669, step=1) a
RangeIndex(start=0, stop=3669, step=1) 더헤어
RangeIndex(start=0, stop=3669, step=1) 뮤직스튜디오
RangeIndex(start=0, stop=3669, step=1) 지1ㅇ
RangeIndex(start=0, stop=3669, step=1) 우리
RangeIndex(start=0, stop=3669, step=1) ĦLA
RangeIndex(start=0, stop=3669, step=1) 폰장
RangeIndex(start=0, stop=3669, step=1) 노래방
RangeIndex(start=0, stop=3669, step=1) 산수
RangeIndex(start=0, stop=3669, step=1) 부동산
RangeIndex(start=0, stop=3669, step=1) KI과
RangeIndex(start=0, stop=3669, step=1) 이마트
RangeIndex(start=0, stop=3669, step=1) 편의점
RangeIndex(start=0, stop=3669, step=1) 허브
RangeIndex(start=0, stop=3669, step=1) 부동산
RangeIndex(start=0, stop=3669, step=1) 1
RangeIndex(start=0, stop

RangeIndex(start=0, stop=3669, step=1) 
RangeIndex(start=0, stop=3669, step=1) 개울가
RangeIndex(start=0, stop=3669, step=1) 셀빨래방
RangeIndex(start=0, stop=3669, step=1) 어학연
RangeIndex(start=0, stop=3669, step=1) 칼ㄹ국수
RangeIndex(start=0, stop=3669, step=1) 
RangeIndex(start=0, stop=3669, step=1) 샤브샤브
RangeIndex(start=0, stop=3669, step=1) 해월궁
RangeIndex(start=0, stop=3669, step=1) 사주
RangeIndex(start=0, stop=3669, step=1) 아
RangeIndex(start=0, stop=3669, step=1) 용두암
RangeIndex(start=0, stop=3669, step=1) 원원
RangeIndex(start=0, stop=3669, step=1) 교육
RangeIndex(start=0, stop=3669, step=1) 방
RangeIndex(start=0, stop=3669, step=1) 한지공예
RangeIndex(start=0, stop=3669, step=1) 손뜨개전문
RangeIndex(start=0, stop=3669, step=1) K
RangeIndex(start=0, stop=3669, step=1) 소화기
RangeIndex(start=0, stop=3669, step=1) 판매
RangeIndex(start=0, stop=3669, step=1) O
RangeIndex(start=0, stop=3669, step=1) 
RangeIndex(start=0, stop=3669, step=1) 아날로그주류상회
RangeIndex(start=0, stop=3669, step=1) 대포
RangeIndex(start=0, st

In [ ]:
submit=pd.read_csv('sample_submission.csv')
submit['text']=result
submit.to_csv('submit.csv', index=False, encoding='utf-8-sig')